<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Converting-qiime2-files" data-toc-modified-id="Converting-qiime2-files-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Converting qiime2 files</a></span><ul class="toc-item"><li><span><a href="#OTU-table" data-toc-modified-id="OTU-table-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>OTU table</a></span></li><li><span><a href="#Taxonomy" data-toc-modified-id="Taxonomy-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Taxonomy</a></span></li><li><span><a href="#Tree" data-toc-modified-id="Tree-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Tree</a></span></li></ul></li><li><span><a href="#Load" data-toc-modified-id="Load-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Formatting-metadata" data-toc-modified-id="Formatting-metadata-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Formatting metadata</a></span></li><li><span><a href="#Checking-overlap" data-toc-modified-id="Checking-overlap-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Checking overlap</a></span></li><li><span><a href="#Making-phyloseq-object" data-toc-modified-id="Making-phyloseq-object-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Making phyloseq object</a></span><ul class="toc-item"><li><span><a href="#Writing-object" data-toc-modified-id="Writing-object-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Writing object</a></span></li></ul></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* Creating a phyloseq object from the merged Qiime2 artifacts

# Var

In [1]:
work_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/'

# all MiSeq runs (LLA_merged)
SV_file = file.path(work_dir, 'table_merged_filt.qza')
tax_file = file.path(work_dir, 'taxonomy.qza')
tree_file = file.path(work_dir, 'aligned-rep-seqs_filt_masked_midroot-tree.qza')

# newest version of metadata
metadata_file = '/ebio/abt3_projects/Georg_animal_feces/data/mapping/unified_metadata_complete_190529.tsv'

# params
conda_env = 'qiime2-2019.10'
threads = 24

# Init

In [4]:
library(dtplyr)
library(dplyr)
library(tidyr)
library(ggplot2)
library(phyloseq)
library(LeyLabRMisc)

In [3]:
df.dims()
make_dir(work_dir)
setwd(work_dir)

Directory already exists: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/ 


# Converting qiime2 files

In [5]:
file.exists(SV_file) %>% print
file.exists(tax_file) %>% print
file.exists(tree_file) %>% print
file.exists(metadata_file) %>% print

[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE


## OTU table

In [6]:
# coverting artifact to biom
cmd = sprintf('qiime tools export --output-path . --input-path %s', SV_file)
bash_job(cmd, conda_env, stderr=TRUE)

Exported /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged//table_merged_filt.qza as BIOMV210DirFmt to directory .

NULL

In [7]:
# converting biom to tsv
cmd = sprintf('biom convert -i feature-table.biom -o otu_table.tsv --to-tsv')
bash_job(cmd, conda_env, stderr=TRUE)

NULL

## Taxonomy

In [8]:
# coverting artifact to biom
cmd = sprintf('qiime tools export --output-path . --input-path %s', tax_file)
bash_job(cmd, conda_env, stderr=TRUE)

Exported /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged//taxonomy.qza as TSVTaxonomyDirectoryFormat to directory .

NULL

In [9]:
# formatting taxonomy
exe = '/ebio/abt3_projects/Georg_animal_feces/code/taxonomy_format.py'
cmd = sprintf('%s %s > %s', exe, 'taxonomy.tsv', 'taxonomy_L0-6.tsv')
bash_job(cmd, conda_env, stderr=TRUE)

NULL

## Tree

In [10]:
# coverting artifact to biom
cmd = sprintf('qiime tools export --output-path . --input-path %s', tree_file)
bash_job(cmd, conda_env, stderr=TRUE)

Exported /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged//aligned-rep-seqs_filt_masked_midroot-tree.qza as NewickDirectoryFormat to directory .

NULL

# Load

In [11]:
# otu table
physeq_otu = data.table::fread('otu_table.tsv', sep='\t', skip=1) %>%
    as.data.frame
colnames(physeq_otu)[1] = 'X.OTU.ID'
rownames(physeq_otu) = physeq_otu$X.OTU.ID
physeq_otu$X.OTU.ID = NULL
colnames(physeq_otu) = gsub(' $', '', colnames(physeq_otu))

physeq_otu 

,F14_Common_Bream,F35_Red_Deer,F36_Red_Deer,F44_Fallow_Deer,F45_Red_Deer,F46_Red_Deer,F47_Red_Deer,F48_Red_Deer,F53_Mouflon,F66_Wild_Boar,⋯,X354_Hoffmanns_Two_toed_Sloth,X357_Northern_Tamandua,X359_Brown_throated_Sloth,X4_Red_Fox,X403_Chacma_Baboon,X405_Nile_Crocodile,X431_Quagga,X44_Meadow_Viper,X67_European_Roe,X98_Eurasian_Red_Squirrel
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
77f55e61d7f58d90f3ef64b3606594e2,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
7227f0b58d14954b49687a6fc68e5bf6,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
a088965958efffb207a1168f48649e0e,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
efc922cdf4d0824db9bd0f23e6b5ab1f,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [12]:
# metadata
metadata = read.delim(metadata_file, sep='\t') %>%
    mutate(SampleID = gsub('^([0-9])', 'X\\1', SampleID)) %>%
    as.data.frame

metadata 

SampleID,host_subject_id,host_german_name,host_common_name,scientific_name,scientific_name_subspecies,host_taxid,class,order,family,⋯,temp_regulation,collection_date,season,samp_store_days,samp_collect_device,sampling_method,diet,feeding_style,digestive_system,digestive_system_detail
<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
F14_Common_Bream,F14,Brachse,Common.Bream,Abramis_brama,NA,38527,Actinopterygii,Cypriniformes,Cyprinidae,⋯,ectotherm/poikilotherm,2010-08-10,summer,317,Greiner Polypropylene Tube 50ml,feces sampling,omnivore,NA,simple,simple
F35_Red_Deer,F35,Rothirsch,Red.Deer,Cervus_elaphus,NA,9860,Mammalia,Artiodactyla,Cervidae,⋯,endotherm/homoiotherm,2010-09-29,autumn,229,Greiner Polystyrene 30ml Faeces Container,feces sampling,herbivore,intermediate type,foregut,foregut
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X189_fail_Rainbow_Trout,189.fail,Regenbogenforelle,Rainbow.Trout,Oncorhynchus_mykiss,NA,8022,Actinopterygii,Salmoniformes,Salmonidae,⋯,ectotherm/poikilotherm,2012-03-01,winter,467,Eppendorf tube,feces sampling,carnivore,NA,simple,simple
X232_fail_Brown_Trout,232.fail,Seeforelle,Brown.Trout,Salmo_trutta,NA,8032,Actinopterygii,Salmoniformes,Salmonidae,⋯,ectotherm/poikilotherm,2012-03-15,winter,453,Polyethylene freezer bag,gut section,carnivore,NA,simple,simple


In [13]:
# taxonomy 
physeq_tax = read.delim('taxonomy_L0-6.tsv', sep='\t') %>%
    filter(Feature.ID %in% rownames(physeq_otu))
rownames(physeq_tax) = physeq_tax$Feature.ID
physeq_tax$Feature.ID = NULL
physeq_tax = physeq_tax %>% 
    dplyr::select(-Confidence)

physeq_tax 

,Domain,Phylum,Class,Order,Family,Genus,Species
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
000029244345a7c6e454809d6a334492,Bacteria,Bacteroidetes,Bacteroidia,unclassified,unclassified,unclassified,unclassified
000102e42b17dbeeaec2c42e1c5a600c,Bacteria,Firmicutes,Clostridia,Clostridiales,Ruminococcaceae,unclassified,unclassified
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
fffd4d88f98e37e62eaa23deafeca2d0,Bacteria,Firmicutes,Clostridia,Clostridiales,Lachnospiraceae,unclassified,unclassified
ffffb69ce78f6f3888ccc1b61f829077,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Prevotellaceae,Prevotellaceae UCG-001,bacterium P3


In [14]:
# tree 
tree = ape::read.tree('tree.nwk')
tree


Phylogenetic tree with 81276 tips and 81157 internal nodes.

Tip labels:
	27056924d25a972043773840061213f9, 91783a64f15025a663fb7ca42b62aa27, bf7812de16984aabb5ff7b218905d930, 0fe0885c69de8bf86dd8d74a49cc9d78, e33d2aee347a38209febc0579b219495, b9c6d5d5c82259dfb90ce105f6ad03da, ...
Node labels:
	root, 0.864, 0.496, 0.975, 0.875, 1.000, ...

Rooted; includes branch lengths.

# Formatting metadata

In [15]:
# striping whitespace from labels
tws = function(x){
    if(class(x) == 'numeric'){
        return(x)
    } else {
        x = sapply(as.character(x), function(y) trimws(y, which = 'both'))
        return(x)
    }
}


metadata = metadata %>%
    mutate_all(tws) %>%
    as.data.frame
    
rownames(metadata) = metadata$SampleID
metadata 

,SampleID,host_subject_id,host_german_name,host_common_name,scientific_name,scientific_name_subspecies,host_taxid,class,order,family,⋯,temp_regulation,collection_date,season,samp_store_days,samp_collect_device,sampling_method,diet,feeding_style,digestive_system,digestive_system_detail
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
F14_Common_Bream,F14_Common_Bream,F14,Brachse,Common.Bream,Abramis_brama,NA,38527,Actinopterygii,Cypriniformes,Cyprinidae,⋯,ectotherm/poikilotherm,2010-08-10,summer,317,Greiner Polypropylene Tube 50ml,feces sampling,omnivore,NA,simple,simple
F35_Red_Deer,F35_Red_Deer,F35,Rothirsch,Red.Deer,Cervus_elaphus,NA,9860,Mammalia,Artiodactyla,Cervidae,⋯,endotherm/homoiotherm,2010-09-29,autumn,229,Greiner Polystyrene 30ml Faeces Container,feces sampling,herbivore,intermediate type,foregut,foregut
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X189_fail_Rainbow_Trout,X189_fail_Rainbow_Trout,189.fail,Regenbogenforelle,Rainbow.Trout,Oncorhynchus_mykiss,NA,8022,Actinopterygii,Salmoniformes,Salmonidae,⋯,ectotherm/poikilotherm,2012-03-01,winter,467,Eppendorf tube,feces sampling,carnivore,NA,simple,simple
X232_fail_Brown_Trout,X232_fail_Brown_Trout,232.fail,Seeforelle,Brown.Trout,Salmo_trutta,NA,8032,Actinopterygii,Salmoniformes,Salmonidae,⋯,ectotherm/poikilotherm,2012-03-15,winter,453,Polyethylene freezer bag,gut section,carnivore,NA,simple,simple


In [16]:
# adding extra rows to metadata
x = data.frame(Sample = colnames(physeq_otu)) %>%
    filter(!is.na(Sample))

metadata_j = metadata %>%
    right_join(x, c('SampleID'='Sample')) %>%
    as.data.frame

rownames(metadata_j) = metadata_j$SampleID
metadata_j 

Warning message:
“Column `SampleID`/`Sample` joining character vector and factor, coercing into character vector”


,SampleID,host_subject_id,host_german_name,host_common_name,scientific_name,scientific_name_subspecies,host_taxid,class,order,family,⋯,temp_regulation,collection_date,season,samp_store_days,samp_collect_device,sampling_method,diet,feeding_style,digestive_system,digestive_system_detail
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
F14_Common_Bream,F14_Common_Bream,F14,Brachse,Common.Bream,Abramis_brama,NA,38527,Actinopterygii,Cypriniformes,Cyprinidae,⋯,ectotherm/poikilotherm,2010-08-10,summer,317,Greiner Polypropylene Tube 50ml,feces sampling,omnivore,NA,simple,simple
F35_Red_Deer,F35_Red_Deer,F35,Rothirsch,Red.Deer,Cervus_elaphus,NA,9860,Mammalia,Artiodactyla,Cervidae,⋯,endotherm/homoiotherm,2010-09-29,autumn,229,Greiner Polystyrene 30ml Faeces Container,feces sampling,herbivore,intermediate type,foregut,foregut
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X67_European_Roe,X67_European_Roe,67,Reh,European.Roe,Capreolus_capreolus,NA,9858,Mammalia,Artiodactyla,Cervidae,⋯,endotherm/homoiotherm,2011-10-31,autumn,#VALUE!,Screw cap vial,feces sampling,herbivore,concentrate selector,foregut,foregut
X98_Eurasian_Red_Squirrel,X98_Eurasian_Red_Squirrel,98,Eichhoernchen,Eurasian.Red.Squirrel,Sciurus_vulgaris,NA,55149,Mammalia,Rodentia,Sciuridae,⋯,endotherm/homoiotherm,2011-11-03,autumn,#VALUE!,Screw cap vial,gut section,omnivore,NA,simple hindgut,caecum


# Checking overlap

In [17]:
overlap(rownames(metadata), colnames(physeq_otu))
setdiff(rownames(metadata), colnames(physeq_otu)) %>% length %>% print
setdiff(colnames(physeq_otu), rownames(metadata)) %>% length %>% print

intersect(x,y): 280 
setdiff(x,y): 195 
setdiff(y,x): 21 
union(x,y): 496 
[1] 195
[1] 21


In [18]:
setdiff(colnames(physeq_otu), rownames(metadata))

[1] "methanogen"      "mock"            "Neg_control_1_A" "Neg_control_2_B"
 [5] "Neg_control_3_C" "Neg_control_4_D" "Neg_control_5_E" "Neg_control_6_F"
 [9] "Neg_control_7_G" "Neg_control_8_H" "Neg_control_9_I" "Pos_control_1_J"
[13] "Pos_control_2_K" "Pos_control_3_L" "Pos_control_4_M" "Pos_control_5_N"
[17] "Pos_control_6_O" "Pos_control_7_P" "Pos_control_8_Q" "Pos_control_9_R"
[21] "water"

In [19]:
overlap(colnames(physeq_otu), rownames(metadata_j))

intersect(x,y): 301 
setdiff(x,y): 0 
setdiff(y,x): 0 
union(x,y): 301 


In [20]:
overlap(rownames(physeq_otu), rownames(physeq_tax))

intersect(x,y): 81276 
setdiff(x,y): 0 
setdiff(y,x): 0 
union(x,y): 81276 


In [21]:
overlap(rownames(physeq_otu), tree$tip.label)

intersect(x,y): 81276 
setdiff(x,y): 0 
setdiff(y,x): 0 
union(x,y): 81276 


# Making phyloseq object

* all samples in the dataset
* all taxa in the dataset

In [22]:
physeq = phyloseq(
                  otu_table(physeq_otu %>% as.matrix, taxa_are_rows=TRUE),
                  sample_data(metadata_j),
                  tax_table(physeq_tax %>% as.matrix),
                  phy_tree(tree)
                 )
physeq

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 81276 taxa and 301 samples ]
sample_data() Sample Data:       [ 301 samples by 71 sample variables ]
tax_table()   Taxonomy Table:    [ 81276 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 81276 tips and 81157 internal nodes ]

## Writing object

In [23]:
# output dir
D = file.path(work_dir, 'phyloseq')
make_dir(D)

Created directory: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged//phyloseq 


In [24]:
# writing
physeq_file = file.path(D, 'physeq_all.RDS')
saveRDS(physeq, file=physeq_file)
cat('File written:', physeq_file)

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged//phyloseq/physeq_all.RDS

# sessionInfo

In [25]:
sessionInfo()

R version 3.6.2 (2019-12-12)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.3 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Georg_animal_feces/envs/phyloseq/lib/libopenblasp-r0.3.7.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] LeyLabRMisc_0.1.1 phyloseq_1.30.0   ggplot2_3.2.1     tidyr_1.0.0      
[5] dplyr_0.8.3       dtplyr_1.0.0     

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.3          ape_5.3             lattice_0.20-38    
 [4] Biostrings_2.54.0   assertthat_0.2.1    zeallot_0.1.0